In [1]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.7/404.7 kB 12.5 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import FunctionTransformer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import optuna

In [3]:
train = pd.read_csv('/content/train_enhanced_features.csv')

test = pd.read_csv('/content/test_enhanced_features.csv')

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [5]:
train.set_index('tpep_pickup_datetime', inplace=True)
test.set_index('tpep_pickup_datetime', inplace=True)

In [6]:
train.drop(columns=['lag_1', 'lag_2', 'lag_3', 'lag_4'], inplace=True)
test.drop(columns=['lag_1', 'lag_2', 'lag_3', 'lag_4'], inplace=True)

In [7]:
cont_cols = ['avg_pickups', 'pickups_same_time_yesterday', 'last_4_days_std']

log_transformer = FunctionTransformer(np.log1p)
log_transformer.set_output(transform='pandas')

transformer = ColumnTransformer(
    transformers=[
        ('scaler', StandardScaler(), cont_cols),
        ('minmax', log_transformer, ['total_pickups'])
    ],
    remainder='passthrough'
)
transformer.set_output(transform='pandas')

train = transformer.fit_transform(train)
test = transformer.transform(test)

In [8]:
col_name = [col.split('__', 1)[1] for  col in train.columns]
train.columns = col_name
test.columns = col_name

In [9]:
train = pd.concat([train['total_pickups'], train.drop(columns=['total_pickups'])], axis=1)
test = pd.concat([test['total_pickups'], test.drop(columns=['total_pickups'])], axis=1)

In [10]:
class CustomDataset(Dataset):
  def __init__(self, dataframe, sequence_length, num_features_count, cat_features_count):
    self.df = dataframe.values
    self.sequence_length = sequence_length
    self.num_features_count = num_features_count
    self.cat_features_count = cat_features_count

  def __len__(self):
    return len(self.df) - self.sequence_length

  def __getitem__(self, idx):

    chunk = self.df[idx: idx + self.sequence_length]
    target = self.df[idx + self.sequence_length, 0]

    x_num = torch.tensor(chunk[:, : self.num_features_count], dtype=torch.float32)
    x_cat = torch.tensor(chunk[:, self.num_features_count : ], dtype=torch.long)

    return x_num, x_cat, target

In [11]:
class MixedInputLSTM(nn.Module):

  def __init__(self, num_numerical_cols, categorical_cardinalities, embedding_dims, hidden_size):
    super(MixedInputLSTM, self).__init__()

    self.embeddings = nn.ModuleList([
        nn.Embedding(num_embeddings=counts, embedding_dim=dim)
        for counts, dim in zip(categorical_cardinalities, embedding_dims)
    ])

    self.total_input_size = num_numerical_cols + sum(embedding_dims)
    self.lstm = nn.LSTM(input_size=self.total_input_size, hidden_size = hidden_size, batch_first=True)

    self.fc = nn.Linear(hidden_size, 1)

  def forward(self, x_numerical, x_categorical):
    embedding_list = []
    for i, emb_layer in enumerate(self.embeddings):
      embedding_list.append(emb_layer(x_categorical[:, :, i]))

    x_cat_embedded = torch.cat(embedding_list, dim=2)
    final_input = torch.cat([x_cat_embedded, x_numerical], dim=2)

    lstm_out, (final_hidden_state, final_cell_state) = self.lstm(final_input)
    out = self.fc(final_hidden_state.squeeze(0)) # Corrected: Use squeeze(0) to get (batch_size, hidden_size)
    return out

In [12]:
class SMAPE(nn.Module):
  def __init__(self):
    super().__init__()

  def forward(self, pred, target):
    pred = torch.expm1(pred)
    target = torch.expm1(target)
    abs_diff = torch.abs(pred-target)
    denominator = (torch.abs(pred) + torch.abs(target)) / 2
    diff = abs_diff / denominator
    return torch.mean(diff)

In [13]:
def objective(trial):
  print(f'\n\nStarting Trial #{trial.number}')
  epochs = trial.suggest_int('num_epochs', 30, 100)
  learning_rate = trial.suggest_categorical('lr', [0.01, 0.001, 0.0001])
  optimizer_name = trial.suggest_categorical('optim', ['Adam', 'RMSprop', 'SGD'])
  hidden_layers = trial.suggest_categorical('hidden_layers', [32, 64, 128, 256])
  seq_length = trial.suggest_int('seq', 4, 96)
  batch_size = trial.suggest_int('batch', 100, 200)

  loss_function = SMAPE()
  model = MixedInputLSTM(num_numerical_cols=4, categorical_cardinalities=[30, 7, 24, 2], embedding_dims=[15, 4, 12, 1], hidden_size=hidden_layers)
  model = model.to(device)

  optimizer_class = getattr(torch.optim, optimizer_name)
  optimizer = optimizer_class(model.parameters(),
                             lr=learning_rate)

  num_cols = 4 # fixed
  cat_cols = 4 # fixed

  traindataset = CustomDataset(train, seq_length, num_cols, cat_cols)
  testdataset = CustomDataset(test, seq_length, num_cols, cat_cols)

  traindataloader = DataLoader(traindataset, batch_size=batch_size, shuffle=False)
  testdataloader = DataLoader(testdataset, batch_size=batch_size, shuffle=False)

  best_val_score = float('inf')

  for epoch in range(epochs):

    model.train()
    epoch_loss = []

    for numbatchx, catbatchx, batchy in traindataloader:
      numbatchx, catbatchx, batchy = numbatchx.to(device), catbatchx.to(device), batchy.to(device)
      pred = model(numbatchx, catbatchx)
      loss = loss_function(pred, batchy)

      with torch.no_grad():
        epoch_loss.append(loss.item())

      optimizer.zero_grad()

      loss.backward()

      optimizer.step()

    val_loss = []
    model.eval()
    with torch.no_grad():
      for numbatchx, catbatchx, batchy in testdataloader:
        numbatchx, catbatchx, batchy = numbatchx.to(device), catbatchx.to(device), batchy.to(device)
        prediction = model(numbatchx, catbatchx)
        score = loss_function(prediction, batchy)
        val_loss.append(score.item())

    avg_val_score = np.mean(val_loss)

    if avg_val_score < best_val_score:
      best_val_score = avg_val_score

    print(f'in {epoch} epoch of trial {trial.number} the loss is {np.mean(val_loss)}')
    trial.report(np.mean(val_loss), epoch)
    if trial.should_prune():
      raise optuna.exceptions.TrialPruned()

  return best_val_score

In [ ]:
pruner = optuna.pruners.MedianPruner(n_startup_trials=5, n_warmup_steps=10)
study = optuna.create_study(direction='minimize', pruner=pruner)
study.optimize(objective, n_trials = 100)

[I 2026-01-16 18:58:36,908] A new study created in memory with name: no-name-b1735d22-9fb4-4813-a37d-80b2bb27ba9f




Starting Trial #0
in 0 epoch of trial 0 the loss is 0.8205779388318014
in 1 epoch of trial 0 the loss is 0.7438374129849642
in 2 epoch of trial 0 the loss is 0.6611993905663113
in 3 epoch of trial 0 the loss is 0.6707683039959969
in 4 epoch of trial 0 the loss is 0.6403161717156763
in 5 epoch of trial 0 the loss is 0.6242752742348395
in 6 epoch of trial 0 the loss is 0.622712809306713
in 7 epoch of trial 0 the loss is 0.6258324233502747
in 8 epoch of trial 0 the loss is 0.6312449474299379
in 9 epoch of trial 0 the loss is 0.6292009611882533
in 10 epoch of trial 0 the loss is 0.6197842876474852
in 11 epoch of trial 0 the loss is 0.6206548324360304
in 12 epoch of trial 0 the loss is 0.6186335948058894
in 13 epoch of trial 0 the loss is 0.6063942110785498
in 14 epoch of trial 0 the loss is 0.6048438166339801
in 15 epoch of trial 0 the loss is 0.6018652476332225
in 16 epoch of trial 0 the loss is 0.6007922018625043
in 17 epoch of trial 0 the loss is 0.5935569293837952
in 18 epoch of tria

[I 2026-01-16 19:07:01,191] Trial 0 finished with value: 0.5487095321578038 and parameters: {'num_epochs': 40, 'lr': 0.0001, 'optim': 'RMSprop', 'hidden_layers': 256, 'seq': 41, 'batch': 160}. Best is trial 0 with value: 0.5487095321578038.


in 39 epoch of trial 0 the loss is 0.5487095321578038


Starting Trial #1
in 0 epoch of trial 1 the loss is 1.0448817289080998
in 1 epoch of trial 1 the loss is 1.0268363866969692
in 2 epoch of trial 1 the loss is 0.9796219157791032
in 3 epoch of trial 1 the loss is 0.9505817665980946
in 4 epoch of trial 1 the loss is 0.9037927429456173
in 5 epoch of trial 1 the loss is 0.8840205079679504
in 6 epoch of trial 1 the loss is 0.8638301736074168
in 7 epoch of trial 1 the loss is 0.8346628680049711
in 8 epoch of trial 1 the loss is 0.7916262210388366
in 9 epoch of trial 1 the loss is 0.7529976235886672
in 10 epoch of trial 1 the loss is 0.7312717131182693
in 11 epoch of trial 1 the loss is 0.7228436420084846
in 12 epoch of trial 1 the loss is 0.6803189811791698
in 13 epoch of trial 1 the loss is 0.6321705701591593
in 14 epoch of trial 1 the loss is 0.615694928311378
in 15 epoch of trial 1 the loss is 0.600747814847046
in 16 epoch of trial 1 the loss is 0.5842164678794348
in 17 epoch of trial

[I 2026-01-16 19:14:46,250] Trial 1 finished with value: 0.5738137739051645 and parameters: {'num_epochs': 38, 'lr': 0.01, 'optim': 'Adam', 'hidden_layers': 64, 'seq': 72, 'batch': 113}. Best is trial 0 with value: 0.5487095321578038.


...

in 10 epoch of trial 24 the loss is 0.5968594901624622


Starting Trial #25
in 0 epoch of trial 25 the loss is 0.8365973191494517
in 1 epoch of trial 25 the loss is 0.7293661971125635
in 2 epoch of trial 25 the loss is 0.6418084911660903
in 3 epoch of trial 25 the loss is 0.6396619789017819
in 4 epoch of trial 25 the loss is 0.6321071863808694
in 5 epoch of trial 25 the loss is 0.6128133211972902
in 6 epoch of trial 25 the loss is 0.6061197350385449
in 7 epoch of trial 25 the loss is 0.5934388338794006
in 8 epoch of trial 25 the loss is 0.5945373873659938
in 9 epoch of trial 25 the loss is 0.5804758898270733
in 10 epoch of trial 25 the loss is 0.5655237414521914
in 11 epoch of trial 25 the loss is 0.5610005169795702
in 12 epoch of trial 25 the loss is 0.5653692706943108
in 13 epoch of trial 25 the loss is 0.555491359771035
in 14 epoch of trial 25 the loss is 0.5525219738226153
in 15 epoch of trial 25 the loss is 0.5492627057846876
in 16 epoch of trial 25 the loss is 0.546442427850894
i

[I 2026-01-16 22:39:41,892] Trial 25 finished with value: 0.530452859232007 and parameters: {'num_epochs': 92, 'lr': 0.001, 'optim': 'RMSprop', 'hidden_layers': 128, 'seq': 46, 'batch': 109}. Best is trial 17 with value: 0.5300940658829408.


in 91 epoch of trial 25 the loss is 0.5313297370662907


Starting Trial #26
in 0 epoch of trial 26 the loss is 0.719214622954909
in 1 epoch of trial 26 the loss is 0.6609446884158258
in 2 epoch of trial 26 the loss is 0.6191126900393166
in 3 epoch of trial 26 the loss is 0.5799536758875437
in 4 epoch of trial 26 the loss is 0.573848848767589
in 5 epoch of trial 26 the loss is 0.5618356312157203
in 6 epoch of trial 26 the loss is 0.5555853044518019
in 7 epoch of trial 26 the loss is 0.5505770545024925
in 8 epoch of trial 26 the loss is 0.5458094629013958
in 9 epoch of trial 26 the loss is 0.5449078471285214
in 10 epoch of trial 26 the loss is 0.5428834167440341
in 11 epoch of trial 26 the loss is 0.5416224891123785
in 12 epoch of trial 26 the loss is 0.541639293775449
in 13 epoch of trial 26 the loss is 0.5380309538600755
in 14 epoch of trial 26 the loss is 0.5370874322080695
in 15 epoch of trial 26 the loss is 0.5377545814697249
in 16 epoch of trial 26 the loss is 0.535313039544283
in 

[I 2026-01-16 22:57:25,694] Trial 26 finished with value: 0.530099842612806 and parameters: {'num_epochs': 92, 'lr': 0.001, 'optim': 'RMSprop', 'hidden_layers': 32, 'seq': 59, 'batch': 110}. Best is trial 17 with value: 0.5300940658829408.


in 91 epoch of trial 26 the loss is 0.5337438920866124


Starting Trial #27
in 0 epoch of trial 27 the loss is 0.7894839881333867
in 1 epoch of trial 27 the loss is 0.6464570761728302
in 2 epoch of trial 27 the loss is 0.5856724846279839
in 3 epoch of trial 27 the loss is 0.5627999947504565
in 4 epoch of trial 27 the loss is 0.5531121561597994
in 5 epoch of trial 27 the loss is 0.5580743668494684
in 6 epoch of trial 27 the loss is 0.545644645376601
in 7 epoch of trial 27 the loss is 0.5442011641259472
in 8 epoch of trial 27 the loss is 0.5415626563390826
in 9 epoch of trial 27 the loss is 0.5386196794662415
in 10 epoch of trial 27 the loss is 0.5375866242696141
in 11 epoch of trial 27 the loss is 0.5372911962650868
in 12 epoch of trial 27 the loss is 0.5356492230680082
in 13 epoch of trial 27 the loss is 0.534977827598439
in 14 epoch of trial 27 the loss is 0.5357008244049736
in 15 epoch of trial 27 the loss is 0.5347495169961299
in 16 epoch of trial 27 the loss is 0.5368662618268393
i

In [ ]:
study.best_params

In [ ]:
study.best_value